## AuxTel AzEl offsets - 27-Apr-21

In this notebook, investigate az-el offsets from 11-Mar-21
Re-doing this with different calculation of theta

In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pickle as pkl
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord, AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from lsst.daf.butler import Butler as gen3Butler
from lsst.daf.persistence import Butler as gen2Butler
from lsst_efd_client import EfdClient
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
infile = open('/project/cslage/AuxTel/offsets/offsets_16apr21.pkl','rb')
charVisits = pkl.load(infile)
infile.close()

In [ ]:
# This helps make the plots more compact
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

In [ ]:
# Gen3 butler
dayObs = 20210311
expId = 2021031100422
REPO_DIR = '/repo/main'
butler = gen3Butler(REPO_DIR, collections="LATISS/raw/all")
mData = butler.get('raw.metadata', detector=0, exposure=expId)
for key in mData.keys():
    print(key, mData[key])


In [ ]:
# Set Cerro Pachon location and observation time
location = EarthLocation.from_geodetic(lon=mData['OBS-LONG']*u.deg,
                                       lat=mData['OBS-LAT']*u.deg,
                                       height=mData['OBS-ELEV']*u.m)

utcoffset = -3*u.hour  
time = Time(mData['DATE-BEG']) + utcoffset
time.format = 'iso'

In [ ]:
print(location.lat.deg, time)

In [ ]:
def rotAngle(mData, time, location):
    rotpa = Angle(mData['ROTPA']*u.deg)
    ra = Angle(mData['RASTART']*u.deg)
    dec = Angle(mData['DECSTART']*u.deg)
    coords = SkyCoord(ra=ra, dec=dec)
    altaz = coords.transform_to(AltAz(obstime=time,location=location, \
                                              pressure=mData['PRESSURE']*u.torr, temperature=mData['AIRTEMP']*u.deg_C, \
                                             relative_humidity=mData['HUMIDITY']*u.percent))
    sinTheta = np.cos(location.lat.rad) * np.sin(altaz.az.rad) / np.cos(dec.rad)
    theta = Angle(np.arcsin(sinTheta)*u.rad)
    print(theta.deg, rotpa.deg)
    return [theta, rotpa]

In [ ]:
rotAngle(mData, time, location)

In [ ]:
# Pick an expId, and compare this image with the next one in the sequence.
myExpId = 2021031100422
for charVisit in charVisits:
    expId = charVisit['Visit'][0]
    if expId == myExpId:
        break
nextExpId = myExpId + 1
for nextCharVisit in charVisits:
    thisExpId = nextCharVisit['Visit'][0]
    if thisExpId == nextExpId:
        break
cat = charVisit['brightCatalog']
nextCat = nextCharVisit['brightCatalog']
# These are the measured shifts between the two catalogs
shift_x = nextCharVisit['brightestCentroid'][0] - charVisit['brightestCentroid'][0]
shift_y = nextCharVisit['brightestCentroid'][1] - charVisit['brightestCentroid'][1] 
exp = charVisit['exp']
nextExp = nextCharVisit['exp']
rotpa = charVisit['Visit'][6]
# These are the commanded offsets in Az, El
off_az = nextCharVisit['Visit'][7] - charVisit['Visit'][7]
off_el = nextCharVisit['Visit'][8] - charVisit['Visit'][8]

# Now put off_az and off_el in pixels, and rotate them using rotpa
off_az /= exp.getWcs().getPixelScale().asArcseconds()
off_el /= exp.getWcs().getPixelScale().asArcseconds()

off = np.array([off_az, off_el])
theta = Angle(36.8*u.deg).rad # This is what it needs to be
#theta = rotAngle(mData, time, location)[0].rad
c, s = np.cos(theta), np.sin(theta)
# This is the rotation matrix that puts the commanded offsets into the detector coordinates
R = np.array(((c, -s), (s, c))) 
rotated_off = R.dot(off)

# Now plot it all
plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.title(f"Image - {myExpId}",fontsize=18)
arr = exp.image.array
arr = np.clip(arr, 1, 100000) # This image has some negative values, and this removes them
img = plt.imshow(arr, norm=LogNorm(vmin=1, vmax=1000),  interpolation='Nearest', cmap='gray')
plt.scatter(cat['base_SdssCentroid_x'],cat['base_SdssCentroid_y']\
            ,color='red', marker='x', label="Measured")
plt.arrow(charVisit['brightestCentroid'][0],charVisit['brightestCentroid'][1], rotated_off[0], rotated_off[1],\
            color='green', width = 20, label='Commanded offset')
plt.arrow(charVisit['brightestCentroid'][0],charVisit['brightestCentroid'][1], shift_x, shift_y,\
            color='red', width=20, label='Measured offset')
plt.xlim(0,4000)
plt.ylim(4000,0)
colorbar(img)
plt.legend()

plt.subplot(1,2,2)
plt.title(f"Image - {nextExpId}",fontsize=18)
nextArr = nextExp.image.array
nextArr = np.clip(nextArr, 1, 100000) # This image has some negative values, and this removes them
img = plt.imshow(nextArr, norm=LogNorm(vmin=1, vmax=1000),  interpolation='Nearest', cmap='gray')
plt.scatter(nextCat['base_SdssCentroid_x'],nextCat['base_SdssCentroid_y']\
            ,color='red', marker='x', label="Measured")
plt.scatter(cat['base_SdssCentroid_x'] + rotated_off[0],cat['base_SdssCentroid_y'] + rotated_off[1]\
            ,color='green', marker='+', s=200, label="Expected")
plt.xlim(0,4000)
plt.ylim(4000,0)
colorbar(img)
plt.legend()

plt.tight_layout(h_pad=1)
#plt.savefig(f"/project/cslage/AuxTel/offsets/Offsets_Meas_vs_Expected_{myExpId}_19Apr21.pdf")

In [ ]:
# Plot the problem
expId = 2021031100422
for charVisit in charVisits:
    visit = charVisit['Visit']
    expId = visit[0]
    if expId == myExpId:
        break
az = visit[9]
dec = -89.30854#visit[12]

r = 90.0 + dec
az_angle = 180.0 - az
zenith = -90.0 - location.lat.deg
dx = (zenith + 1.0) * np.tan(az_angle*np.pi / 180.0)

fig = plt.figure(figsize = (16,8))
plt.suptitle(f"Header values in expId {expId} are inconsistent", fontsize = 24)
ax1 = plt.subplot(1,2,1,aspect='equal')
decCircle = plt.Circle((0,0), r, fill=False)
poleCircle = plt.Circle((0,0), 0.05, fill=True)
ax1.add_patch(decCircle)
ax1.add_patch(poleCircle)
ax1.plot([0.0,0.0], [zenith, 1.0])
ax1.plot([0.0,dx], [zenith, 1.0])
ax1.text(0.1,0.2, "SCP", fontsize=12)
ax1.text(0.0,1.2, "Meridian", fontsize=12)
ax1.text(-1.8,1.2, "Azimuth = %.5f"%az, fontsize=12)
ax1.text(0.8,0.0, "Dec = %.5f"%dec, fontsize=12)
ax1.set_xlim(-2.0,2.0)
ax1.set_ylim(-2.0,2.0)
ax2 = plt.subplot(1,2,2,aspect='equal')
decCircle = plt.Circle((0,0), r, fill=False)
poleCircle = plt.Circle((0,0), 0.05, fill=True)
ax2.add_patch(decCircle)
ax2.add_patch(poleCircle)
ax2.plot([0.0,0.0], [zenith, 1.0])
ax2.plot([0.0,dx], [zenith, 1.0])
ax2.text(2.0,-1.0, "SCP", fontsize=12)
ax2.text(2.0,zenith, "Zenith", fontsize=12)
ax2.set_xlim(-36.0,36.0)
ax2.set_ylim(-70.0,2.0)
plt.savefig(f"/project/cslage/AuxTel/offsets/Header_Issue_{expId}_20Apr21.pdf")

In [ ]:
# Plot the problem
expId = 2021031100422
for charVisit in charVisits:
    visit = charVisit['Visit']
    myExpId = visit[0]
    if expId == myExpId:
        break
az = visit[9]
dec = visit[12]

r = 90.0 + dec
az_angle = 180.0 - az
zenith = 90.0 + location.lat.deg
dx = -(zenith + 1.0) * np.tan(az_angle*np.pi / 180.0)
dy = -location.lat.deg
fig = plt.figure(figsize = (16,8))
plt.suptitle(f"Header values in expId {expId} are inconsistent", fontsize = 24)
ax1 = plt.subplot(1,2,1,aspect='equal')
decCircle = plt.Circle((0,dy), r, fill=False)
poleCircle = plt.Circle((0,dy), 0.05, fill=True)
ax1.add_patch(decCircle)
ax1.add_patch(poleCircle)
ax1.plot([0.0,0.0], [zenith+dy, -1.0+dy])
ax1.plot([0.0,dx], [zenith+dy, -1.0+dy])
ax1.text(0.1,-0.2+dy, "SCP", fontsize=12)
ax1.text(0.0,-1.2+dy, "Meridian", fontsize=12)
ax1.text(-1.8,-1.2+dy, "Azimuth = %.5f"%az, fontsize=12)
ax1.text(0.8,0.0+dy, "Dec = %.5f"%dec, fontsize=12)
ax1.set_xlim(-2.0,2.0)
ax1.set_ylim(2.0+dy,-2.0+dy)
ax1.set_ylabel("Altitude")
ax2 = plt.subplot(1,2,2,aspect='equal')
decCircle = plt.Circle((0,dy), r, fill=False)
poleCircle = plt.Circle((0,dy), 0.05, fill=True)
ax2.add_patch(decCircle)
ax2.add_patch(poleCircle)
ax2.plot([0.0,0.0], [zenith+dy, -1.0+dy])
ax2.plot([0.0,dx], [zenith+dy, -1.0+dy])
ax2.text(2.0,1.0+dy, "SCP", fontsize=12)
ax2.text(2.0,zenith+dy, "Zenith", fontsize=12)
ax2.set_xlim(-36.0,36.0)
ax2.set_ylim(65.0+dy,-2.0+dy)
ax2.set_ylabel("Altitude")
plt.savefig(f"/project/cslage/AuxTel/offsets/Header_Issue_{expId}_20Apr21.pdf")